In [64]:
#If you need to install in modules in jupyter notebook 
%pip install scikit-learn
%pip install pandas
%pip install matplotlib 
%pip install seaborn 
%pip install graphviz
%pip install scikit-plot   
%pip install statsmodels   
%pip install keras
%pip install tensorflow

#Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)
import datetime # Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"


# Import required libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

# Keras specific
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical 

# to make this notebook's output stable across runs
np.random.seed(42)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "decision_trees"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)

os.makedirs(IMAGES_PATH, exist_ok=True)
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):

    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\shaun\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\shaun\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pillow>=6.2.0 in c:\users\shaun\appdata\local\programs\python\python37\lib\site-packages (from matplotlib) (7.2.0)


You should consider upgrading via the 'c:\users\shaun\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


...

You should consider upgrading via the 'c:\users\shaun\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\shaun\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


  Attempting uninstall: setuptools
    Found existing installation: setuptools 39.0.1
    Uninstalling setuptools-39.0.1:
      Successfully uninstalled setuptools-39.0.1
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\shaun\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [65]:
#changes the output for the print statements
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth',0)

In [78]:
#Pull 
df = pd.read_csv('https://raw.githubusercontent.com/Rodrig79/Machine-Learning-Data-Analysis-Project/master/rawData/pokemon.csv') 

In [79]:
from sklearn import preprocessing

#Data Cleaning
df = df[~df.Name.str.contains('Mega')] #removed pokemon with "Mega" in it
df = df.drop(columns = ["Name","Type 1","Type 2","#","Generation"]) #Removed columns with names

In [80]:
print(df.shape)
df.describe()

(751, 8)


,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
count,751.000000,751.000000,751.000000,751.000000,751.000000,751.000000,751.000000
mean,423.322237,68.669774,75.978695,71.744341,70.213049,70.109188,66.607190
std,112.932936,25.946152,29.920554,29.629870,30.263860,27.401096,27.845601
min,180.000000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000
25%,325.000000,50.000000,55.000000,50.000000,45.000000,50.000000,45.000000
50%,430.000000,65.000000,75.000000,68.000000,65.000000,65.000000,65.000000
75%,500.000000,80.000000,95.000000,90.000000,90.000000,85.000000,88.500000
max,770.000000,255.000000,180.000000,230.000000,180.000000,230.000000,180.000000


In [81]:
target_column = ['Legendary']
predictors = list(set(list(df.columns))-set(target_column))
df[predictors] = df[predictors]/df[predictors].max()
df.describe()

,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
count,751.000000,751.000000,751.000000,751.000000,751.000000,751.000000,751.000000
mean,0.549769,0.269293,0.422104,0.311932,0.390072,0.304823,0.370040
std,0.146666,0.101750,0.166225,0.128826,0.168133,0.119135,0.154698
min,0.233766,0.003922,0.027778,0.021739,0.055556,0.086957,0.027778
25%,0.422078,0.196078,0.305556,0.217391,0.250000,0.217391,0.250000
50%,0.558442,0.254902,0.416667,0.295652,0.361111,0.282609,0.361111
75%,0.649351,0.313725,0.527778,0.391304,0.500000,0.369565,0.491667
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [82]:
X = df[predictors].values
y = df[target_column].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
print(X_train.shape); print(X_test.shape)

(525, 7)
(226, 7)


In [83]:
# one hot encode outputs
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

count_classes = y_test.shape[1]
print(count_classes)

2


In [84]:
model = Sequential()
model.add(Dense(500, activation='relu', input_dim=7))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [85]:
# build the model
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
17/17 [==============================] - 0s 2ms/step - loss: 0.4782 - accuracy: 0.9105
Epoch 2/20
17/17 [==============================] - 0s 2ms/step - loss: 0.3859 - accuracy: 0.9124
Epoch 3/20
17/17 [==============================] - 0s 1ms/step - loss: 0.3475 - accuracy: 0.9124
Epoch 4/20
17/17 [==============================] - 0s 1ms/step - loss: 0.3251 - accuracy: 0.9124
Epoch 5/20
17/17 [==============================] - 0s 1ms/step - loss: 0.2881 - accuracy: 0.9124
Epoch 6/20
17/17 [==============================] - 0s 1ms/step - loss: 0.2440 - accuracy: 0.9124
Epoch 7/20
17/17 [==============================] - 0s 1ms/step - loss: 0.1880 - accuracy: 0.9295
Epoch 8/20
17/17 [==============================] - 0s 1ms/step - loss: 0.1585 - accuracy: 0.9448
Epoch 9/20
17/17 [==============================] - 0s 1ms/step - loss: 0.1212 - accuracy: 0.9581
Epoch 10/20
17/17 [==============================] - 0s 1ms/step - loss: 0.1168 - accuracy: 0.9600
Epoch 11/20
17/17 [

In [87]:
pred_train= model.predict(X_train)
scores = model.evaluate(X_train, y_train, verbose=0)
print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores[1], 1 - scores[1]))   
 
pred_test= model.predict(X_test)
scores2 = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))

Accuracy on training data: 0.977142870426178% 
 Error on training data: 0.02285712957382202
Accuracy on test data: 0.9646017551422119% 
 Error on test data: 0.035398244857788086


In [144]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(2, input_dim=1, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Model: "sequential_41"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_98 (Dense)             (None, 2)                 4         
_________________________________________________________________
dense_99 (Dense)             (None, 1)                 3         
Total params: 7
Trainable params: 7
Non-trainable params: 0
_________________________________________________________________
None


In [182]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model
model = Sequential()
model.add(Dense(2, input_dim=1, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [185]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\shaun\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [186]:
def tf.keras.utils.plot_model(
    model,
    to_file="model.png",
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
)


NameError: name 'tf' is not defined

In [184]:
dot_img_file = '/tmp/model_1.png'
tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)

NameError: name 'tf' is not defined

In [169]:
import graphviz